# ETL DB writers

> Pre-specified nodes that write data into the database

In [ ]:
#| default_exp etl_db_writers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import pandas as pd
import psycopg2
import typing as t
from tqdm import tqdm

import os
import csv
from tqdm import tqdm
import shutil


from inventory_foundation_sdk.db_mgmt import get_db_credentials

[01/19/25 19:29:08] INFO     Using                                                                  ]8;id=307715;file:///Users/magnus/miniforge3/envs/inventory_foundation/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=590847;file:///Users/magnus/miniforge3/envs/inventory_foundation/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/magnus/miniforge3/envs/inventory_foundation/lib/python3.11/sit                
                             e-packages/kedro/framework/project/rich_logging.yml' as logging                       
                             configuration.                                                                        

### Company-level

In [ ]:
#| export

import logging
logger = logging.getLogger(__name__)

def write_company_name(name: str, additional_info: t.Dict = None, ignore_company_if_exist: bool = True) -> int:
     
    """
    This function writes the company name to the database and any additional info.
    Each key in `additional_info` becomes a column in the database table if it doesn't exist,
    and the associated value is written to that column.
    
    If `ignore_company_if_exist` is False and the company name already exists, an error is raised.
    If `ignore_company_if_exist` is True, a warning is logged and the existing record is updated if additional info differs.
    
    Returns the ID that the database has assigned to the company name.
    """

    db_credentials = get_db_credentials()["con"]

    try:
        with psycopg2.connect(db_credentials) as conn:
            with conn.cursor() as cur:
                # Try to insert the company name
                cur.execute(
                    """
                    INSERT INTO companies (name)
                    VALUES (%s)
                    ON CONFLICT (name) DO NOTHING
                    RETURNING "ID";
                    """,
                    (name,)
                )
                result = cur.fetchone()
                
                if result is None:
                    # Company exists, handle based on ignore_company_if_exist flag
                    cur.execute(
                        """
                        SELECT "ID" FROM companies WHERE name = %s;
                        """,
                        (name,)
                    )
                    company_id = cur.fetchone()[0]
                    
                    if not ignore_company_if_exist:
                        raise ValueError(f"Company '{name}' already exists.")
                    
                    logger.warning("Company already exists, ignoring new entry")

                    # Check if additional info needs to be updated
                    if additional_info is not None:
                        for key, value in additional_info.items():
                            # Add column if it doesn't exist
                            cur.execute(
                                f"""
                                ALTER TABLE companies
                                ADD COLUMN IF NOT EXISTS {key} TEXT;
                                """
                            )
                            
                            # Check current value before updating
                            cur.execute(
                                f"""
                                SELECT {key} FROM companies WHERE "ID" = %s;
                                """,
                                (company_id,)
                            )
                            current_value = cur.fetchone()[0]
                            
                            # Only update if the value is different
                            if current_value != value:
                                logger.warning(f"Overwriting '{key}' for company '{name}' from '{current_value}' to '{value}'.")
                                cur.execute(
                                    f"""
                                    UPDATE companies
                                    SET {key} = %s
                                    WHERE "ID" = %s;
                                    """,
                                    (value, company_id)
                                )
                else:
                    company_id = result[0]
                    
                    # Insert additional information for new entry
                    if additional_info is not None:
                        for key, value in additional_info.to_dict().items():
                            # Add column if it doesn't exist
                            cur.execute(
                                f"""
                                ALTER TABLE companies
                                ADD COLUMN IF NOT EXISTS {key} TEXT;
                                """
                            )
                            # Update the row with the additional information
                            cur.execute(
                                f"""
                                UPDATE companies
                                SET {key} = %s
                                WHERE "ID" = %s;
                                """,
                                (value, company_id)
                            )
                    
    except Exception as e:
        logger.error(f"Got error while writing company name to database: {e}")
        raise e

    return company_id

### Category level
#### Main function to write categories

In [ ]:
#| export

def write_categories(categories: dict, company_id: int, category_level_names: list = None) -> t.List[int]:
    
    """
    This function writes the categories to the database.

    The categories are expected to be in a dictionary with the following structure:
    {1:
        {
            "category_name": None
        }
    2:
        {
            "category_name": ["parent_category_1", "parent_category_2", ...]
        }
    ...
    }

    For the first level, the parent list should be None.

    If a category on a lower level has another parent from 2 or more levels above, the cateogry should be
    listed under the lowest level parent. (such that the write db function can first write all parents and then the children)
    """

    if category_level_names is not None:
        write_category_level_descriptions(category_level_names, company_id)

    for i, level in categories.items():
        write_category_level(level, company_id)

    return True
   

#### Helper functions for categories

In [ ]:
#| export

def write_category_level_descriptions(category_level_names, company_id):
        
    """
    This function writes the names of the category levels to the database.
    """

    db_credentials = get_db_credentials()["con"]

    try:
        with psycopg2.connect(db_credentials) as conn:
            with conn.cursor() as cur:
                for i, name in enumerate(category_level_names):
                    cur.execute(
                        """
                        INSERT INTO category_level_descriptions ("companyID", level, name)
                        VALUES (%s, %s, %s)
                        ON CONFLICT ("companyID", level) 
                        DO UPDATE SET name = EXCLUDED.name;
                        """,
                        (company_id, i+1, name)
                    )
    except Exception as e:
        logger.error(f"Got error while writing category level names to database: {e}")
        raise e

def write_category_level(categories: list, company_id: int) -> t.Dict[str, int]:
    
    """
    This function writes one level of categories into the database and returns a list of the IDs that the database has assigned.
    The purpose is to call this function repeatedly for each level of categories.

    It will add data to two tables:
     - `categories` with the category names and a flag if it is a leaf category
     - `category_relations` with the parent-child relationships between categories

    """

    db_credentials = get_db_credentials()["con"]

    try:
        with psycopg2.connect(db_credentials) as conn:
            with conn.cursor() as cur:
                for key, value in categories.items():
                    parents = value
                    cur.execute(
                        """
                        WITH ins AS (
                            INSERT INTO categories ("companyID", name)
                            VALUES (%s, %s)
                            ON CONFLICT ("companyID", name) DO NOTHING
                            RETURNING "ID"
                        )
                        SELECT "ID" FROM ins
                        UNION ALL
                        SELECT "ID" FROM categories
                        WHERE "companyID" = %s AND name = %s
                        LIMIT 1;
                        """,
                        (company_id, key, company_id, key)
                    )
                    category_id = cur.fetchone()[0]
                    
                    if parents is not None:
                        
                        for parent in parents:
                            cur.execute(
                                """
                                SELECT "ID" FROM categories
                                WHERE "companyID" = %s AND name = %s;
                                """,
                                (company_id, parent)
                            )
                            parent_id = cur.fetchone()[0]
                            cur.execute(
                                """
                                INSERT INTO category_relations ("subID", "parentID")
                                VALUES (%s, %s)
                                ON CONFLICT ("subID", "parentID") 
                                DO NOTHING;
                                """,
                                (category_id, parent_id)
                            )
        return

    except Exception as e:
        logger.error(f"Got error while writing category level names to database: {e}")
        raise e

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()